## Credit Score Project

In [2]:
#importing necessary libaries 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import matplotlib.pyplot as plt
#from dataprep.clean import clean_headers

In [6]:
import polars as pl 

ModuleNotFoundError: No module named 'polars'

In [3]:
#Importing the dataset for train
train_path = Path('data', 'data_devsample.csv')
credit_df = pd.read_csv(train_path)

In [ ]:
df = pl.read_csv(train_path)
print(df.shape)
df.head()

In [4]:
credit_df.shape

(80000, 197)

In [11]:
credit_df.sort_values(by=)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_ACTIVE,MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_CLOSED,MEAN_AMTCR_OVERDUE_0M_12M_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_0M_12M_TYPE_EQ_CLOSED,FIRST_TRANSACTION_TIME_MONTHS,LAST_TRANSACTION_TIME_MONTHS,TIME,BASE,DAY,MONTH
0,215138,0.0,Cash loans,M,N,Y,0.0,135000.0,646920.0,25195.5,...,0.0,NaN,0.0,NaN,37.0,28.0,2018-06-23,1,20180623,201806
1,325963,0.0,Cash loans,M,Y,Y,0.0,135000.0,740218.5,40284.0,...,0.0,0.0,0.0,NaN,3.0,18.0,2018-06-16,1,20180616,201806
2,206923,0.0,Cash loans,F,N,Y,0.0,180000.0,339241.5,12919.5,...,0.0,NaN,0.0,NaN,73.0,61.0,2018-06-19,1,20180619,201806
3,265855,0.0,Cash loans,F,N,Y,0.0,180000.0,1264428.0,37098.0,...,0.0,0.0,NaN,NaN,36.0,4.0,2018-08-09,1,20180809,201808
4,359795,0.0,Revolving loans,F,N,Y,NaN,76500.0,202500.0,10125.0,...,NaN,NaN,0.0,0.0,12.0,12.0,2018-11-13,1,20181113,201811


In [ ]:
cnums_df = credit_df[nums_list:=[col for col in credit_df.columns if credit_df[col].dtypes in [int,float]]]
cat_df = credit_df.drop(nums_list, axis= 'columns')

In [ ]:
print((cat_df.isnull().sum() > 0).sum())
cat_df.isna().sum()

6


name_contract_type                0
code_gender                       0
flag_own_car                      0
flag_own_realty                   0
name_type_suite                 353
name_income_type                  0
name_education_type               0
name_family_status                0
name_housing_type                 0
occupation_type               25067
weekday_appr_process_start        0
organization_type                 0
fondkapremont_mode            54886
housetype_mode                40120
wallsmaterial_mode            40626
emergencystate_mode           37892
time                              0
dtype: int64

In [ ]:
missing_col = [col for col in cat_df.columns if cat_df[col].isna().sum()]
for col in missing_col:
    print(f"{col} unique values are: {cat_df[col].unique()} \n with {cat_df[col].isnull().sum()} missing rows \n")

name_type_suite unique values are: ['Family' 'Unaccompanied' 'Spouse, partner' 'Children' 'Group of people'
 nan 'Other_B' 'Other_A'] 
 with 353 missing rows 

occupation_type unique values are: ['Laborers' nan 'Medicine staff' 'Drivers' 'Sales staff' 'Managers'
 'Accountants' 'Cooking staff' 'Waiters/barmen staff' 'Core staff'
 'High skill tech staff' 'Cleaning staff' 'HR staff' 'Security staff'
 'Private service staff' 'Low-skill Laborers' 'Secretaries'
 'Realty agents' 'IT staff'] 
 with 25067 missing rows 

fondkapremont_mode unique values are: [nan 'reg oper account' 'org spec account' 'reg oper spec account'
 'not specified'] 
 with 54886 missing rows 

housetype_mode unique values are: ['block of flats' nan 'specific housing' 'terraced house'] 
 with 40120 missing rows 

wallsmaterial_mode unique values are: ['Panel' nan 'Stone, brick' 'Monolithic' 'Others' 'Block' 'Wooden' 'Mixed'] 
 with 40626 missing rows 

emergencystate_mode unique values are: ['No' nan 'Yes'] 
 with 37892 

0               Family
1        Unaccompanied
2        Unaccompanied
3        Unaccompanied
4        Unaccompanied
             ...      
79995    Unaccompanied
79996    Unaccompanied
79997          Other_A
79998    Unaccompanied
79999    Unaccompanied
Name: name_type_suite, Length: 80000, dtype: object